In [13]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from keras.layers import Input, Dense, Dropout
from keras.models import Model, Sequential
from sklearn.metrics import roc_curve, auc,roc_auc_score
from sklearn.model_selection import train_test_split
import pandas as pd
from matplotlib import gridspec
from scipy import stats

## References
[Paper](https://arxiv.org/pdf/2009.02205.pdf) <br>
[Data](https://zenodo.org/record/4287694) <br>
[Repo](https://github.com/bnachman/DCTRHunting/blob/master/CWoLaSimulationAssistedDecorrelation.ipynb) <br>


# DATA

In [14]:
f = h5py.File("data/anomaly_detection/events_anomalydetection_v2.features.h5")
print(f.keys())
f.close()

<KeysViewHDF5 ['df']>


In [15]:
filePath = "data/anomaly_detection/events_anomalydetection_v2.features.h5"
data = pd.read_hdf(filePath, key='df')
data.head()

,pxj1,pyj1,pzj1,mj1,tau1j1,tau2j1,tau3j1,pxj2,pyj2,pzj2,mj2,tau1j2,tau2j2,tau3j2,label
0,-1467.239990,611.502014,511.101990,38.896000,8.290660,4.836080,4.260190,1403.579956,-674.551025,-451.670990,237.893997,79.815102,21.010300,16.757601,0.0
1,-1211.239990,347.315002,547.963013,389.532013,191.804001,99.562798,70.872200,619.341003,-62.177299,-1944.040039,22.999201,8.042190,6.335090,5.525360,0.0
2,-1229.619995,649.857971,8.089170,72.155502,47.168098,37.243198,33.658199,1196.250000,-647.896973,-1283.109985,78.230698,15.292900,13.944200,10.013500,0.0
3,-693.304016,-1046.729980,1716.910034,55.797798,24.788601,6.890150,5.813390,747.961975,994.250000,-412.966003,359.113007,175.209000,103.500999,84.447098,0.0
4,-1488.199951,-25.370100,-30.989700,84.891502,26.878799,15.517200,13.260400,1415.640015,20.905100,223.630997,77.506500,57.986000,34.147400,26.660601,0.0


```diff
background (1m) and 2-prong signal (100k)
clustered every event into R=1 jets using the anti-kT algorithm.

four features: m(j1), m(j2)-m(j1), τ21 (j1), τ21 (j2)
    the invariant mass of the lighter jet
    the mass difference of the leading two jets
    the N-subjettiness τ21 of the leading two jets.
extra features: mjj, label

SR = signal region
SB = sideband
```

In [16]:
data_4feats = pd.DataFrame()
data_4feats['m_j1'] = data['mj1']
data_4feats['m_j2-m_j1'] = data['mj2']-data['mj1']
data_4feats['tau21_j1'] = data['tau2j1'] / data['tau1j1']
data_4feats['tau21_j2'] = data['tau2j2'] / data['tau1j2']
data_4feats['mjj'] = (((data["pxj1"]**2+data["pyj1"]**2+data["pzj1"]**2+data["mj1"]**2)**0.5+(data["pxj2"]**2+data["pyj2"]**2+data["pzj2"]**2+data["mj2"]**2)**0.5)**2-(data["pxj1"]+data["pxj2"])**2-(data["pyj1"]+data["pyj2"])**2-(data["pzj1"]+data["pzj2"])**2)**0.5/1000.
data_4feats['label'] = data['label']

# the dependence between the jet masses and mjj is artificially strengthened by redefining  mj → mj + α*mjj for α = 0.1.
data_4feats['m_j1'] = data_4feats['m_j1'] + 0.1 * data_4feats['mjj']
data_4feats.head()

,m_j1,m_j2-m_j1,tau21_j1,tau21_j2,mjj,label
0,39.226722,198.997997,0.583317,0.263237,3.307219,0.0
1,389.842775,-366.532812,0.519086,0.787732,3.107621,0.0
2,72.455992,6.075195,0.789584,0.911809,3.004895,0.0
3,56.121106,303.315208,0.277956,0.590729,3.233075,0.0
4,85.183437,-7.385002,0.577303,0.588890,2.919346,0.0


In [17]:
sig = data_4feats[data_4feats['label']==1].to_numpy()[:,:-1]
bg = data_4feats[data_4feats['label']==0].to_numpy()[:,:-1]

In [18]:
# 1500 signal for training
sig_inject = sig[:1500]
sig_inject2 = sig[1500:3000]
sig_test = sig[1500:]
# half background for training                                  # bg1 - Pythia, bg2 - Herwig
bg_inject = bg[:int(bg.shape[0]*.5)]
bg_test = bg[int(bg.shape[0]*.5):]

In [19]:
def getSR(data,SR_low,SR_high):
    return data[(data[:,-1]>SR_low)*(data[:,-1]<SR_high)][:,:4]
def getSB(data,SR_low,SR_high,SB_low,SB_high):
    left = data[(data[:,-1]>SB_low)*(data[:,-1]<SR_low)][:,:4]
    right = data[(data[:,-1]>SB_low)*(data[:,-1]<SR_low)][:,:4]
    return left, right

In [20]:
# signal region (mjj)
SR_low = 3.3
SR_high = 3.7

sig_SR_train = getSR(sig_inject,SR_low,SR_high)
sig_SR_test = getSR(sig_test,SR_low,SR_high)
bg_SR_train = getSR(bg_inject,SR_low,SR_high)                   
bg_SR_test = getSR(bg_test,SR_low,SR_high)                           # should also have bg2 part       

# sideband (mjj)
SB_low = 3.1
SB_high = 3.9

sig_SB_low,sig_SB_high = getSB(sig_inject,SR_low,SR_high,SB_low,SB_high)                    # inject + bg        
bg_SB_low,bg_SB_high = getSB(bg_inject,SR_low,SR_high,SB_low,SB_high)

sig_SB_low_test,sig_SB_high_test = getSB(sig_inject2,SR_low,SR_high,SB_low,SB_high)         # inject2 + bg2
bg_SB_low_test,bg_SB_high_test = getSB(bg_test,SR_low,SR_high,SB_low,SB_high)                   


In [21]:
X_train = np.concatenate([sig_SR_train,bg_SR_train,sig_SB_train,bg_SB_train])
X_train = np.concatenate([sig_SR_train,bg_SR_train,sig_SB_low,bg_SB_low,sig_SB_high,bg_SB_high])
Y_train = np.concatenate([np.ones(sig_SR_train.shape[0]),np.ones(sig_SB_train.shape[0]),np.zeros(bg_SR_test.shape[0]),np.zeros(bg_SR_test.shape[0])])

X_test = np.concatenate([sig_SR_test,bg_SR_test])
Y_test = np.concatenate([np.ones(sig_SR_test.shape[0]),np.zeros(bg_SR_test.shape[0])])

# MODEL

In [14]:
# CWoLa
model_cwola = Sequential(name="CWoLA")
model_cwola.add(Dense(64, input_dim=4, activation='relu')) 
model_cwola.add(Dense(64, activation='relu'))
model_cwola.add(Dense(64, activation='relu'))
model_cwola.add(Dense(1, activation='sigmoid'))
model_cwola.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cwola.summary()

Model: "CWoLA"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 64)                320       
_________________________________________________________________
dense_22 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_23 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 65        
Total params: 8,705
Trainable params: 8,705
Non-trainable params: 0
_________________________________________________________________


In [15]:
# simulation-augmented CWoLA
model_sacwola = Sequential(name="SA CWoLA")
model_sacwola.add(Dense(64, input_dim=4, activation='relu')) 
model_sacwola.add(Dense(64, activation='relu'))
model_sacwola.add(Dense(64, activation='relu'))
model_sacwola.add(Dense(1, activation='sigmoid'))
model_sacwola.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_sacwola.summary()

Model: "SA CWoLA"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 64)                320       
_________________________________________________________________
dense_26 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_27 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 65        
Total params: 8,705
Trainable params: 8,705
Non-trainable params: 0
_________________________________________________________________
